# Graph

---

## Create the Graph Schema
### Setting Up the Environment Variables
You can either use the following code or set them directly using the `export` command in your shell.

In [1]:
import os
os.environ["TG_HOST"] = "http://127.0.0.1"
os.environ["TG_USERNAME"] = "tigergraph"
os.environ["TG_PASSWORD"] = "tigergraph"

### Create Schema

In [2]:
graph_schema = {
    "graph_name": "Social",
    "nodes": {
        "Person": {
            "primary_key": "name",
            "attributes": {
                "name": "STRING",
                "age": "UINT",
                "gender": "STRING",
            },
        },
    },
    "edges": {
        "Friendship": {
            "is_directed_edge": False,
            "from_node_type": "Person",
            "to_node_type": "Person",
            "attributes": {
                "closeness": "DOUBLE",
            },
        },
    },
}

In [3]:
from tigergraphx import Graph
G = Graph(graph_schema)

2025-05-26 22:06:00,084 - tigergraphx.core.managers.schema_manager - INFO - Creating schema for graph: Social...
2025-05-26 22:06:02,856 - tigergraphx.core.managers.schema_manager - INFO - Graph schema created successfully.


---

## Query Operations
### Get Edges

In [4]:
>>> G = Graph(graph_schema)
>>> persons = [
...     ("Alice", {"age": 30, "gender": "Female"}),
...     ("Bob", {"age": 28, "gender": "Male"}),
...     ("Carol", {"age": 32, "gender": "Female"}),
... ]
>>> G.add_nodes_from(persons, "Person")

3

In [5]:
>>> friendships = [
...     ("Alice", "Bob", {"closeness": 0.8}),
...     ("Bob", "Carol", {"closeness": 0.6}),
...     ("Alice", "Carol", {"closeness": 0.9}),
... ]
>>> G.add_edges_from(friendships, "Person", "Friendship", "Person")

3

In [6]:
>>> df = G.get_edges(edge_types="Friendship")
>>> print(df)

       s      t
0    Bob  Carol
1    Bob  Alice
2  Carol    Bob
3  Carol  Alice
4  Alice    Bob
5  Alice  Carol


In [7]:
>>> df = G.get_edges(
...     edge_types="Friendship",
...     filter_expression="e.closeness > 0.7",
... )
>>> print(df)

       s      t
0    Bob  Alice
1  Carol  Alice
2  Alice    Bob
3  Alice  Carol


In [8]:
>>> df = G.get_edges(
...     edge_types="Friendship",
...     return_attributes=["closeness"],
... )
>>> print(df)

       s      t  closeness
0    Bob  Carol        0.6
1    Bob  Alice        0.8
2  Carol    Bob        0.6
3  Carol  Alice        0.9
4  Alice    Bob        0.8
5  Alice  Carol        0.9


In [9]:
>>> df = G.get_edges(
...     edge_types="Friendship",
...     limit=2,
... )
>>> print(df)

     s      t
0  Bob  Carol
1  Bob  Alice


In [10]:
>>> df = G.get_edges(
...     edge_types="Friendship",
...     source_node_alias="s",
...     edge_alias="e",
...     target_node_alias="t",
...     filter_expression='s.gender == "Female" and t.age > 30',
... )
>>> print(df)

       s      t
0  Alice  Carol


In [11]:
>>> G.clear()

True

### create/install/run/drop query

In [12]:
>>> G = Graph(graph_schema)
>>> nodes = [
...     ("Alice", {"age": 30, "gender": "Female"}),
...     ("Bob", {"age": 32, "gender": "Male"}),
...     ("Carol", {"age": 29, "gender": "Female"}),
... ]
>>> G.add_nodes_from(nodes, "Person")

3

In [13]:
>>> edges = [
...     ("Alice", "Bob", {"closeness": 2.0}),
...     ("Bob", "Carol", {"closeness": 3.0}),
... ]
>>> G.add_edges_from(edges)

2

In [14]:
>>> gsql_query = f'''
... CREATE QUERY getFriends(VERTEX<Person> person) FOR GRAPH Social {{
...     Start = {{person}};
...     Friends = SELECT tgt FROM Start:s -(Friendship:e)->:tgt;
...     PRINT Friends;
... }}
... '''
>>> G.create_query(gsql_query)

True

In [15]:
>>> G.install_query("getFriends")

2025-05-26 22:06:18,378 - tigergraphx.core.managers.query_manager - INFO - Installing query 'getFriends' for graph 'Social'...
2025-05-26 22:06:59,268 - tigergraphx.core.managers.query_manager - INFO - Query 'getFriends' installed successfully.


True

In [16]:
>>> result = G.run_query("getFriends", {"person": "Alice"})
>>> print(result)

[{'Friends': [{'v_id': 'Bob', 'v_type': 'Person', 'attributes': {'name': 'Bob', 'age': 32, 'gender': 'Male'}}]}]


In [17]:
>>> G.drop_query("getFriends")

True

In [18]:
>>> G.clear()

True

In [19]:
>>> G.drop_graph()

2025-05-26 22:07:21,688 - tigergraphx.core.managers.schema_manager - INFO - Dropping graph: Social...
2025-05-26 22:07:24,424 - tigergraphx.core.managers.schema_manager - INFO - Graph dropped successfully.


---
